In [3]:
import numpy as np
import pandas as pd
import ast
import plotly.express as px
from plotly import graph_objects as go

In [4]:
df=pd.read_csv("flipkart_com-ecommerce_sample.csv")

In [5]:
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [6]:
df.isnull().sum()

uniq_id                       2
crawl_timestamp               2
product_url                   2
product_name                  2
product_category_tree         2
pid                           2
retail_price                 80
discounted_price             80
image                         5
is_FK_Advantage_product       2
description                   4
product_rating                2
overall_rating                2
brand                      5866
product_specifications       16
dtype: int64

In [7]:
df.dropna(subset=['uniq_id', 'crawl_timestamp', 'product_name', 'product_category_tree', 'pid', 'is_FK_Advantage_product', 'product_rating', 'overall_rating'], inplace=True)
df['retail_price'].fillna(df['retail_price'].median(), inplace=True)
df['discounted_price'].fillna(df['discounted_price'].median(), inplace=True)

In [8]:
df.isnull().sum()

uniq_id                       0
crawl_timestamp               0
product_url                   0
product_name                  0
product_category_tree         0
pid                           0
retail_price                  0
discounted_price              0
image                         3
is_FK_Advantage_product       0
description                   2
product_rating                0
overall_rating                0
brand                      5864
product_specifications       14
dtype: int64

## Top products & Top brands

In [10]:
n = 10
top_products = pd.DataFrame(df['product_category_tree'].value_counts()[:n]).reset_index()
top_products.rename(columns={'index': 'Top_Products', 'product_category_tree': 'Total_Count'}, inplace=True)

n=10
top_brands=pd.DataFrame(df['brand'].value_counts()[:n]).reset_index()
top_brands.rename(columns={'index':'Top_Brands','brand':'Total_Count'},inplace=True)

In [11]:
from plotly.subplots import make_subplots

label1=top_products['Top_Products']
value1=top_products['Total_Count']
label2=top_brands['Top_Brands']
value2=top_brands['Total_Count']

fig_both=make_subplots(rows=1,cols=2,specs=[[{'type':'domain'},{'type':'domain'}]])
fig_both.add_trace(go.Pie(labels=label1,values=value1,name="Top Products",pull=[0.3,0,0,0]),1,1)
fig_both.add_trace(go.Pie(labels=label2,values=value2,name="Top Brands",pull=[0.3,0,0,0]),1,2)

fig_both.update_traces(hole=.4,hoverinfo="label+percent+name")
fig_both.update_layout(
    title_text="Top products and brands distribution",
    annotations=[dict(text='Product',x=0.18,y=0.5,font_size=20,showarrow=False),
                 dict(text='Brand',x=0.82,y=0.5,font_size=20,showarrow=False)]
)

## Product Ratings

In [12]:
total_prod=len(df['pid'])
total_ratings=len(df[df['product_rating']!='No rating available'])
top_ratings=len(df[df['product_rating']=='5'])
df_funnel_1=dict(
    number=[total_prod,total_ratings,top_ratings],
    stage=["Top Products","Products with ratings","Products with 5 star rating"]
)
funnel_1_fig=px.funnel(df_funnel_1,x="number",y="stage")
funnel_1_fig.show()